In [1]:
import etl

import pymc as pm
import arviz as az
import pandas as pd

import constants as c
import model as mod
import os
import predict as pred

#### Set up model

In [2]:
df = etl.read_league_season_results(league="M", season=2025)
clean_df, team_ids = etl.preprocess_match_data(raw_data=df)
data_dict, coords = etl.get_data_dict_and_coords(processed_data=clean_df, team_ids=team_ids)

#### Build model

In [3]:
game_model = mod.build_model(data_dict=data_dict, coords=coords)
idata = mod.sample_model(model=game_model)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eta_off, eta_def, theta_off, theta_def, league_eff, league_hfa, sigma_eff, rho_pace, team_pace, league_pace, sigma_poss]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 40 seconds.


#### Sample posterior

In [4]:
post_pred_data = pred.create_posterior_team_pairings(team_ids=team_ids)
post_pred = pred.predict_posterior(model=game_model, trace=idata, new_data=post_pred_data)

Sampling: [away_eff, game_poss, home_eff]


Output()

#### Evaluate posterior

In [ ]:
matchup_preds = pred.get_game_predictions_from_posterior(post_pred=post_pred, team_ids=team_ids)
labeled_preds = etl.add_team_names_to_ids(pred_data=matchup_preds, league='M')
output_preds = pred.format_predictions_for_submission(pred_data=matchup_preds)